### 0.导入包配置GPU

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

import tensorflow as tf 
# config GPU
# tf.debugging.set_log_device_placement(True)
phy_gpus = tf.config.experimental.list_physical_devices('GPU')
print("num of physical gpus: ",len(phy_gpus))
for gpu in phy_gpus:
   tf.config.experimental.set_memory_growth(gpu,True)
 
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras import Sequential

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense,Dropout,Conv2D,MaxPooling2D,Flatten,BatchNormalization
from tensorflow.keras.optimizers import SGD

num of physical gpus:  1


### 1.加载数据

In [2]:
def load_mnist_func(path):
    f = np.load(path)
    x_train, y_train = f['x_train'], f['y_train']
    x_test, y_test = f['x_test'], f['y_test']
    f.close()
    return (x_train, y_train), (x_test, y_test)


(x_train_data,y_train_data),(x_test_data,y_test_data) = load_mnist_func(path='./data/mnist/mnist.npz')
print("x_train_shape:",x_train_data.shape)
print("y_train_shape:",y_train_data.shape)
print("x_test_shape:",x_test_data.shape)
print("x_test_shape:",y_test_data.shape)

x_train_shape: (60000, 28, 28)
y_train_shape: (60000,)
x_test_shape: (10000, 28, 28)
x_test_shape: (10000,)


### 2.数据预处理

In [3]:
# 处理数据
#60000x28x28 ==>60000x784 并且归一化
x_train_data = x_train_data.reshape(-1,28,28,1)/255.0

x_test_data = x_test_data.reshape(-1,28,28,1)/255.0

# ==>onehot
y_train_data = to_categorical(y_train_data,num_classes=10)
y_test_data = to_categorical(y_test_data,num_classes=10)

### 3.搭建CNN模型

In [4]:
model = Sequential()

# conv_1
model.add(Conv2D(input_shape = (28,28,1),filters = 32,kernel_size = 3,
    strides = 1,padding = 'same',activation = 'relu'))
model.add(MaxPooling2D(pool_size = 2,strides =2,padding = 'same',))

# conv_2
model.add(Conv2D(64,3,strides=2,padding='same',activation = 'relu'))
model.add(MaxPooling2D(2,2,'same'))

# conv_3
model.add(Conv2D(128,3,strides=2,padding='same',activation = 'relu'))
model.add(MaxPooling2D(2,2,'same'))

#把第二个池化层的输出扁平化为1维
model.add(Flatten())
model.add(Dense(1024,activation = 'relu'))
model.add(Dropout(0.25))
model.add(Dense(10,activation='softmax'))

# 定义优化器
sgd = SGD(lr=0.01)

# 定义优化器，loss function，训练过程中计算准确率
model.compile(optimizer=sgd,loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 7, 7, 64)          18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 4, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 2, 2, 128)         73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 1, 1, 128)         0         
_________________________________________________________________
flatten (Flatten)            (None, 128)               0

### 4.训练评估模型

In [5]:
model.fit(x_train_data,y_train_data,batch_size=512,epochs=100)

# 评估模型
loss,accuracy = model.evaluate(x_test_data,y_test_data)

model_path="./data/mnist/mnist_cnn.h5"
model.save(model_path,include_optimizer=True,save_format='h5')

print('test loss',loss)
print('test accuracy',accuracy)

Epoch 1/100
118/118 [==============================] - 1s 8ms/step - loss: 2.2964 - accuracy: 0.1303
Epoch 2/100
118/118 [==============================] - 1s 8ms/step - loss: 2.2824 - accuracy: 0.2436
Epoch 3/100
118/118 [==============================] - 1s 8ms/step - loss: 2.2595 - accuracy: 0.3162
Epoch 4/100
118/118 [==============================] - 1s 8ms/step - loss: 2.2069 - accuracy: 0.3504
Epoch 5/100
118/118 [==============================] - 1s 8ms/step - loss: 2.0449 - accuracy: 0.4332
Epoch 6/100
118/118 [==============================] - 1s 8ms/step - loss: 1.5669 - accuracy: 0.5438
Epoch 7/100
118/118 [==============================] - 1s 8ms/step - loss: 1.0828 - accuracy: 0.6560
Epoch 8/100
118/118 [==============================] - 1s 8ms/step - loss: 0.8077 - accuracy: 0.7473
Epoch 9/100
118/118 [==============================] - 1s 8ms/step - loss: 0.6416 - accuracy: 0.8014
Epoch 10/100
118/118 [==============================] - 1s 8ms/step - loss: 0.5368 - accura

118/118 [==============================] - 1s 8ms/step - loss: 0.0726 - accuracy: 0.9776
Epoch 82/100
118/118 [==============================] - 1s 8ms/step - loss: 0.0722 - accuracy: 0.9779
Epoch 83/100
118/118 [==============================] - 1s 8ms/step - loss: 0.0715 - accuracy: 0.9778
Epoch 84/100
118/118 [==============================] - 1s 8ms/step - loss: 0.0707 - accuracy: 0.9787
Epoch 85/100
118/118 [==============================] - 1s 8ms/step - loss: 0.0699 - accuracy: 0.9786
Epoch 86/100
118/118 [==============================] - 1s 8ms/step - loss: 0.0687 - accuracy: 0.9791
Epoch 87/100
118/118 [==============================] - 1s 8ms/step - loss: 0.0695 - accuracy: 0.9782
Epoch 88/100
118/118 [==============================] - 1s 8ms/step - loss: 0.0685 - accuracy: 0.9786
Epoch 89/100
118/118 [==============================] - 1s 8ms/step - loss: 0.0683 - accuracy: 0.9789
Epoch 90/100
118/118 [==============================] - 1s 8ms/step - loss: 0.0682 - accuracy: 

In [6]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

model = load_model("./data/mnist/mnist_cnn.h5")

def pred(file_name,label):
    img = cv2.imread(file_name,0)
    img = cv2.resize(img,(28,28),interpolation = cv2.INTER_NEAREST)
    img = 255-img
    img = img.reshape(-1,28,28,1)/255.0
    result = model.predict(img)
    print("label is ",label,",predict result:",np.argmax(result,axis=1))
    return np.argmax(result,axis=1)

count = 0
right = 0
for img in os.listdir("./data/mnist/test_numbers/"):
    label = img[0]
    result = pred("./data/mnist/test_numbers/"+img,label)
    if str(result[0]) == label:
        right += 1
    count += 1

print("CNN accuracy：",right/count)

label is  0 ,predict result: [0]
label is  1 ,predict result: [1]
label is  2 ,predict result: [2]
label is  3 ,predict result: [3]
label is  4 ,predict result: [4]
label is  5 ,predict result: [5]
label is  6 ,predict result: [5]
label is  7 ,predict result: [3]
label is  8 ,predict result: [8]
label is  9 ,predict result: [9]
CNN accuracy： 0.8
